In [ ]:
import dspy
from dspy.datasets import HotPotQA
from dotenv import load_dotenv
from phoenix.otel import register

load_dotenv()


tracer_provider = register(
  project_name="dspy_optimizer",
  auto_instrument=True
)

dspy.configure(lm=dspy.LM('azure/gpt-4.1-mini'))

def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

trainset = [x.with_inputs('question') for x in HotPotQA(train_seed=2024, train_size=500).train]
react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

tp = dspy.MIPROv2(metric=dspy.evaluate.answer_exact_match, auto="light", num_threads=24)
optimized_react = tp.compile(react, trainset=trainset)
optimized_react.save('optimizer.json')
